In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/vlawhern/arl-eegmodels.git

Cloning into 'arl-eegmodels'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 112 (delta 1), reused 4 (delta 1), pack-reused 106
Receiving objects: 100% (112/112), 80.61 KiB | 2.12 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [ ]:
!pip install mne

In [ ]:
import numpy as np
import pandas as pd
import mne
from mne import read_epochs
from mne import pick_types, Epochs
from mne.channels import read_layout
from tensorflow.keras.utils import to_categorical
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
K.set_image_data_format('channels_last')
import sys, os

# os.rename('/content/arl-eegmodels', '/content/arl_eegmodels')

from arl_eegmodels.EEGModels import EEGNet, ShallowConvNet, DeepConvNet

In [ ]:
# Function to preprocess the data for a specific event pair
def preprocess_data(event_pair_function, epochs_data):
    epochs_data_selected = event_pair_function(epochs_data)

    # Select only gradiometer channels
    grad_channels = mne.pick_types(epochs_data_selected.info, meg='grad')
    grad_channel_names = [epochs_data_selected.ch_names[ch] for ch in grad_channels]
    epochs_data_selected = epochs_data_selected.pick_channels(grad_channel_names)

    return epochs_data_selected

# Your provided functions for each event pair
def hands_vs_feet(epochs_data):
    event_dict = [
        "hand_imagery",
        "feet_imagery"
    ]

    return epochs_data[event_dict]


def hands_vs_word(epochs_data):
    event_dict = [
        "hand_imagery",
        "word_imagery"
    ]

    return epochs_data[event_dict]

def hands_vs_sub(epochs_data):
    event_dict = [
        "hand_imagery",
        "subtraction_imagery"
    ]


    return epochs_data[event_dict]

def feet_vs_word(epochs_data):
    event_dict = [
        "feet_imagery",
        "word_imagery"
    ]

    return epochs_data[event_dict]

def feet_vs_sub(epochs_data):
    event_dict = [
        "feet_imagery",
        "subtraction_imagery"
    ]

    return epochs_data[event_dict]

def word_vs_sub(epochs_data):
    event_dict = [
        "word_imagery",
        "subtraction_imagery"
    ]

    return epochs_data[event_dict]

def min_max_scaler(epoch_data):
    # scales data to -1, 1
    max_abs = np.max(np.abs(epoch_data))
    processed_data = epoch_data / max_abs
    return processed_data

def mne_standard_scaler(epoch_data):
    # Different from sklearn standardscaler
    # Works via channel to channel

    from mne.decoding import Scaler
    scaler = Scaler(scalings="mean")
    scaled_data = scaler.fit_transform(epoch_data)
    return scaled_data

def z_score_scaler(epoch_data):
    # scales by removing mean and unit standard deviation
    mean = np.mean(epoch_data)
    std = np.std(epoch_data)
    processed_data = (epoch_data - mean) / std
    return processed_data

<h1> Using LOOCV for entire epoched dataset with combined runs. <h1>

In [ ]:
# Subject 3

n_epochs = 20

out_f_name = '/content/drive/MyDrive/MEG' +'acc_EEGNet' + '_epochs_' + str(n_epochs) + '_filtered_' + 'sub_7.npy'
print(out_f_name)

# (loading dataset)
print("Loading epoched data...")
epoched_data = mne.read_epochs('/content/drive/MyDrive/MEG/Maxwell Filtered Data/Combined Run data/Sub_3_combined_epochs-epo.fif', preload=True)
print(f"Epoched data successfully loaded {epoched_data}")

/content/drive/MyDrive/MEGacc_EEGNet_epochs_20_filtered_sub_7.npy
Loading epoched data...
Reading /content/drive/MyDrive/MEG/Maxwell Filtered Data/Combined Run data/Sub_3_combined_epochs-epo.fif ...
    Read a total of 4 projection items:
        ECG-planar--0.200-0.400-PCA-01 (1 x 204) active
        ECG-axial--0.200-0.400-PCA-01 (1 x 102) active
        EOG-planar--0.200-0.200-PCA-01 (1 x 204) active
        EOG-axial--0.200-0.200-PCA-01 (1 x 102) active
    Found the data of interest:
        t =    2000.00 ...    6000.00 ms
        0 CTF compensation matrices available
Not setting metadata
400 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 4)
4 projection items activated
Epoched data successfully loaded <EpochsFIF |  400 events (all good), 2 – 6 s, baseline -0.2 – 0 s (baseline period was cropped after baseline correction), ~1.98 GB, data loaded,
 'hand_imagery': 100
 'feet_imagery': 100
 'subtraction_imagery': 100
 'word_imagery'

In [ ]:
event_id = {'hand_imagery': 4, 'feet_imagery': 8, 'subtraction_imagery': 16, 'word_imagery': 32}
event_pairs = [('hand_imagery', 'word_imagery'), ('hand_imagery', 'subtraction_imagery'),
               ('feet_imagery', 'word_imagery'), ('feet_imagery', 'subtraction_imagery')]

nsub = 1

# Motor vs Mental activity
event_pair_1 = [hands_vs_word]
event_pair_2 = [hands_vs_sub]
event_pair_3 = [feet_vs_word]
event_pair_4 = [feet_vs_sub]
event_pair_5 = [hands_vs_feet]
event_pair_6 = [word_vs_sub]

event_pair = [hands_vs_word, hands_vs_sub, feet_vs_word, feet_vs_sub, hands_vs_feet, word_vs_sub]

In [ ]:
# Choosing kernel length values
kl_values = [8,16, 32, 64, 128]

# Controlling dropout values
d_values = [0.1, 0.2, 0.6]

# Initialize the DataFrame to store the results
results_df = pd.DataFrame(columns=['Event Pair', 'Best kl', 'Best d', 'Best AUC Score', 'Best F1 Score'])

# List of event pairs and their corresponding selection functions
event_pairs = [
    ('hands_vs_word', hands_vs_word),
    ('hands_vs_sub', hands_vs_sub),
    ('feet_vs_word', feet_vs_word)
]

# event_pairs = [
#     ('feet_vs_sub', feet_vs_sub),
#     ('hands_vs_feet', hands_vs_feet),
#     ('word_vs_sub', word_vs_sub)
# ]

# Iterate over the event pairs
for event_pair_name, event_pair_function in event_pairs:
    print(f"Training for event pair: {event_pair_name}")

    # Preprocess the data for the current event pair
    epochs_data_selected = preprocess_data(event_pair_function, epoched_data)

    # Extract data and labels
    X = epochs_data_selected.get_data()
    y = epochs_data_selected.events[:, -1]

    # Map the event codes to integer labels starting from 0
    unique_events = np.unique(y)
    event_to_label = {event: idx for idx, event in enumerate(unique_events)}
    y_labels = np.array([event_to_label[event] for event in y])

    # One-hot encode the labels
    y_one_hot = to_categorical(y_labels, num_classes=2)

    # Split the data into training and testing sets (95% for training, 5% for testing)
    X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.05, random_state=42, stratify=y_one_hot)

    # Normalize the data using min-max scaling
    X_train = min_max_scaler(X_train)
    X_test = min_max_scaler(X_test)

    # Reshape the data to match EEGNet input format
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

    # Update the nb_classes parameter in the EEGNet function
    nb_classes = y_train.shape[1]

    # Initialize variables to store the highest AUC score and corresponding kl, d values for the current event pair
    highest_auc_score = 0
    best_kl = None
    best_d = None

    # Iterate over the kl and d values
    for kl in kl_values:
        for d in d_values:
            print()
            print(f"Current kl: {kl}")
            print(f"Current d: {d}")
            print()

            # Configure the EEGNet model
            model = EEGNet(nb_classes=nb_classes, Chans=X_train.shape[1], Samples=X_train.shape[2], dropoutRate=d, kernLength=kl)

            # Compile the model
            model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

            # Set a valid path for model checkpoints
            checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1, save_best_only=True)

            # Class weights
            class_weights = {0: 1, 1: 1}

            # Fit the model to the training data
            history = model.fit(X_train, y_train, batch_size=16, epochs=n_epochs, verbose=2,
                                validation_split=0.2, callbacks=[checkpointer, EarlyStopping(patience=5)],
                                class_weight=class_weights)

            # Evaluate the model on the test data
            y_pred = model.predict(X_test)
            auc_score = roc_auc_score(y_test, y_pred)
            f1_score_val = f1_score(y_test.argmax(axis=1), y_pred.argmax(axis=1), average='macro')

            # Print the AUC and F1 scores
            print('AUC Accuracy Score:', auc_score)
            print('F1 Score:', f1_score_val)

            # Check if the current AUC score is higher than the highest recorded score for the current event pair
            if auc_score > highest_auc_score:
                highest_auc_score = auc_score
                best_kl = kl
                best_d = d

            # Reset the Keras session to clear the model
            K.clear_session()

            print()
            print('=' * 100)
            print('=' * 100)
            print()

    # Append the best parameters and scores for the current event pair to the results DataFrame
    results_df = results_df.append({'Event Pair': event_pair_name, 'Best kl': best_kl, 'Best d': best_d,
                                    'Best AUC Score': highest_auc_score, 'Best F1 Score': f1_score_val},
                                   ignore_index=True)

# Print the results DataFrame
print(results_df)

Training for event pair: hands_vs_word
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Removing projector <Projection | ECG-axial--0.200-0.400-PCA-01, active : True, n_channels : 102, exp. var : 89.40%>
Removing projector <Projection | EOG-axial--0.200-0.200-PCA-01, active : True, n_channels : 102, exp. var : 53.88%>

Current kl: 8
Current d: 0.1

Epoch 1/20

Epoch 1: val_loss improved from inf to 0.68935, saving model to /tmp/checkpoint.h5
10/10 - 12s - loss: 0.6876 - accuracy: 0.5066 - val_loss: 0.6894 - val_accuracy: 0.6579 - 12s/epoch - 1s/step
Epoch 2/20

Epoch 2: val_loss improved from 0.68935 to 0.68458, saving model to /tmp/checkpoint.h5
10/10 - 1s - loss: 0.5503 - accuracy: 0.7237 - val_loss: 0.6846 - val_accuracy: 0.6579 - 993ms/epoch - 99ms/step
Epoch 3/20

Epoch 3: val_loss improved from 0.68458 to 0.67047, saving model to /tmp/checkpoint.h5
10/10 - 1s - loss: 0.4216 - accuracy: 0.8487 - val_loss: 0.6705 - val_accuracy: 0.6579 - 986ms/epoch -

1/1 [==============================] - 0s 119ms/step
AUC Accuracy Score: 0.8800000000000001
F1 Score: 0.3333333333333333



Current kl: 16
Current d: 0.6

Epoch 1/20

Epoch 1: val_loss improved from inf to 0.69506, saving model to /tmp/checkpoint.h5
10/10 - 3s - loss: 0.7148 - accuracy: 0.5197 - val_loss: 0.6951 - val_accuracy: 0.3421 - 3s/epoch - 346ms/step
Epoch 2/20

Epoch 2: val_loss did not improve from 0.69506
10/10 - 1s - loss: 0.6418 - accuracy: 0.6382 - val_loss: 0.6970 - val_accuracy: 0.3421 - 795ms/epoch - 80ms/step
Epoch 3/20

Epoch 3: val_loss did not improve from 0.69506
10/10 - 1s - loss: 0.5922 - accuracy: 0.7500 - val_loss: 0.6983 - val_accuracy: 0.3421 - 815ms/epoch - 82ms/step
Epoch 4/20

Epoch 4: val_loss did not improve from 0.69506
10/10 - 1s - loss: 0.5455 - accuracy: 0.8158 - val_loss: 0.7027 - val_accuracy: 0.3421 - 883ms/epoch - 88ms/step
Epoch 5/20

Epoch 5: val_loss did not improve from 0.69506
10/10 - 1s - loss: 0.4895 - accuracy: 0.8026 - val_loss: 0.7084 

1/1 [==============================] - 0s 145ms/step
AUC Accuracy Score: 0.76
F1 Score: 0.3333333333333333



Current kl: 32
Current d: 0.1

Epoch 1/20

Epoch 1: val_loss improved from inf to 0.69151, saving model to /tmp/checkpoint.h5
10/10 - 4s - loss: 0.6997 - accuracy: 0.5461 - val_loss: 0.6915 - val_accuracy: 0.6579 - 4s/epoch - 421ms/step
Epoch 2/20

Epoch 2: val_loss improved from 0.69151 to 0.68811, saving model to /tmp/checkpoint.h5
10/10 - 1s - loss: 0.5582 - accuracy: 0.7500 - val_loss: 0.6881 - val_accuracy: 0.6579 - 879ms/epoch - 88ms/step
Epoch 3/20

Epoch 3: val_loss improved from 0.68811 to 0.68120, saving model to /tmp/checkpoint.h5
10/10 - 1s - loss: 0.4682 - accuracy: 0.8618 - val_loss: 0.6812 - val_accuracy: 0.6579 - 866ms/epoch - 87ms/step
Epoch 4/20

Epoch 4: val_loss improved from 0.68120 to 0.66938, saving model to /tmp/checkpoint.h5
10/10 - 1s - loss: 0.3715 - accuracy: 0.9276 - val_loss: 0.6694 - val_accuracy: 0.6579 - 872ms/epoch - 87ms/step
Epoch 5/20

Epoch

<ipython-input-6-474665ea5f23>:107: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Event Pair': event_pair_name, 'Best kl': best_kl, 'Best d': best_d,


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Removing projector <Projection | ECG-axial--0.200-0.400-PCA-01, active : True, n_channels : 102, exp. var : 89.40%>
Removing projector <Projection | EOG-axial--0.200-0.200-PCA-01, active : True, n_channels : 102, exp. var : 53.88%>

Current kl: 8
Current d: 0.1

Epoch 1/20

Epoch 1: val_loss improved from inf to 0.69397, saving model to /tmp/checkpoint.h5
10/10 - 3s - loss: 0.7092 - accuracy: 0.4737 - val_loss: 0.6940 - val_accuracy: 0.3684 - 3s/epoch - 343ms/step
Epoch 2/20

Epoch 2: val_loss did not improve from 0.69397
10/10 - 1s - loss: 0.5600 - accuracy: 0.7763 - val_loss: 0.6978 - val_accuracy: 0.3421 - 807ms/epoch - 81ms/step
Epoch 3/20

Epoch 3: val_loss did not improve from 0.69397
10/10 - 1s - loss: 0.4829 - accuracy: 0.7895 - val_loss: 0.7024 - val_accuracy: 0.3421 - 800ms/epoch - 80ms/step
Epoch 4/20

Epoch 4: val_loss did not improve from 0.69397
10/10 - 1s - loss: 0.4145 - accuracy: 0.8487 - v

...

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Removing projector <Projection | ECG-axial--0.200-0.400-PCA-01, active : True, n_channels : 102, exp. var : 89.40%>
Removing projector <Projection | EOG-axial--0.200-0.200-PCA-01, active : True, n_channels : 102, exp. var : 53.88%>

Current kl: 8
Current d: 0.1

Epoch 1/20

Epoch 1: val_loss improved from inf to 0.69493, saving model to /tmp/checkpoint.h5
10/10 - 3s - loss: 0.6946 - accuracy: 0.6184 - val_loss: 0.6949 - val_accuracy: 0.3421 - 3s/epoch - 346ms/step
Epoch 2/20

Epoch 2: val_loss did not improve from 0.69493
10/10 - 1s - loss: 0.5289 - accuracy: 0.8026 - val_loss: 0.6998 - val_accuracy: 0.3421 - 815ms/epoch - 81ms/step
Epoch 3/20

Epoch 3: val_loss did not improve from 0.69493
10/10 - 1s - loss: 0.4271 - accuracy: 0.8618 - val_loss: 0.7056 - val_accuracy: 0.3421 - 818ms/epoch - 82ms/step
Epoch 4/20

Epoch 4: val_loss did not improve from 0.69493
10/10 - 1s - loss: 0.3674 - accuracy: 0.8684 - v

<ipython-input-6-474665ea5f23>:107: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Event Pair': event_pair_name, 'Best kl': best_kl, 'Best d': best_d,


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Removing projector <Projection | ECG-axial--0.200-0.400-PCA-01, active : True, n_channels : 102, exp. var : 89.40%>
Removing projector <Projection | EOG-axial--0.200-0.200-PCA-01, active : True, n_channels : 102, exp. var : 53.88%>

Current kl: 8
Current d: 0.1

Epoch 1/20

Epoch 1: val_loss improved from inf to 0.69495, saving model to /tmp/checkpoint.h5
10/10 - 4s - loss: 0.6850 - accuracy: 0.4934 - val_loss: 0.6950 - val_accuracy: 0.3421 - 4s/epoch - 399ms/step
Epoch 2/20

Epoch 2: val_loss improved from 0.69495 to 0.69429, saving model to /tmp/checkpoint.h5
10/10 - 1s - loss: 0.6073 - accuracy: 0.6513 - val_loss: 0.6943 - val_accuracy: 0.3421 - 865ms/epoch - 87ms/step
Epoch 3/20

Epoch 3: val_loss improved from 0.69429 to 0.69249, saving model to /tmp/checkpoint.h5
10/10 - 1s - loss: 0.5234 - accuracy: 0.8487 - val_loss: 0.6925 - val_accuracy: 0.6053 - 858ms/epoch - 86ms/step
Epoch 4/20

Epoch 4: val_lo

<ipython-input-6-474665ea5f23>:107: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Event Pair': event_pair_name, 'Best kl': best_kl, 'Best d': best_d,


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
